# Advanced Model Tuning

## Preparation
This notebook explores additional capabilities for tuning Eureqa models added in the 2.13 release of the DataRobot API.

Let's start by importing the DataRobot API.  (If you don't have it installed already, you will need to install it in order to run this notebook.)

In [1]:
import datarobot as dr
from datarobot.enums import AUTOPILOT_MODE

## Set Up
Now configure your DataRobot client (unless you're using a configuration file)...

In [2]:
dr.Client(token='<API TOKEN>', endpoint='http://<YOUR ENDPOINT>/api/v2/')

## Create Project with features

Create a new project using the 10K_diabetes dataset. This dataset contains a binary classification on the target `readmitted`.

In [3]:
url = 'https://s3.amazonaws.com/datarobot_public_datasets/10k_diabetes.xlsx'
project = dr.Project.create(url, project_name='10K Advanced Modeling')
print('Project ID: {}'.format(project.id))

Project ID: 5b624d97962d7442b5bdc014


Now, let's set up the project and run Autopilot to get some models.

In [4]:
# Increase the worker count to make the project go faster.
project.set_worker_count(4)

Project(10K Advanced Modeling)

In [5]:
project.set_target('readmitted', mode=AUTOPILOT_MODE.FULL_AUTO)

Project(10K Advanced Modeling)

In [6]:
project.wait_for_autopilot()

In progress: 4, queued: 36 (waited: 0s)
In progress: 4, queued: 36 (waited: 1s)
In progress: 4, queued: 36 (waited: 1s)
In progress: 4, queued: 36 (waited: 2s)
In progress: 4, queued: 36 (waited: 3s)
In progress: 4, queued: 36 (waited: 5s)
In progress: 4, queued: 36 (waited: 9s)
In progress: 4, queued: 35 (waited: 16s)
In progress: 4, queued: 35 (waited: 29s)
In progress: 4, queued: 35 (waited: 50s)
In progress: 4, queued: 35 (waited: 70s)
In progress: 4, queued: 32 (waited: 91s)
In progress: 4, queued: 31 (waited: 111s)
In progress: 4, queued: 31 (waited: 131s)
In progress: 4, queued: 29 (waited: 152s)
In progress: 4, queued: 27 (waited: 172s)
In progress: 4, queued: 27 (waited: 192s)
In progress: 4, queued: 25 (waited: 213s)
In progress: 4, queued: 24 (waited: 233s)
In progress: 4, queued: 22 (waited: 254s)
In progress: 4, queued: 21 (waited: 274s)
In progress: 4, queued: 19 (waited: 295s)
In progress: 4, queued: 18 (waited: 315s)
In progress: 4, queued: 18 (waited: 335s)
In progress

For the purposes of this example, let's look at a Eureqa model.

In [7]:
models = project.get_models()
model = [
    m for m in models
    if m.model_type.startswith('Eureqa Generalized Additive Model')
][0]
model

Model(u'Eureqa Generalized Additive Model Classifier (3000 Generations)')

Now that we have a model, we can start an advanced-tuning session based on that model.

In [8]:
tune = model.start_advanced_tuning_session()

Each model's blueprint consists of a series of tasks.  Each task contains some number of tunable parameters.  Let's take a look at the available (tunable) tasks.

In [9]:
tune.get_task_names()

[u'Eureqa Generalized Additive Model Classifier (3000 Generations)',
 u'Matrix of word-grams occurrences',
 u'One-Hot Encoding']

Let's drill down into the main Eureqa task, to see what parameters it has available.

In [10]:
task_name = 'Eureqa Generalized Additive Model Classifier (3000 Generations)'
tune.get_parameter_names(task_name)

[u'XGB_subsample',
 u'EUREQA_timeout_sec',
 u'EUREQA_split_mode',
 u'EUREQA_building_block__subtraction',
 u'XGB_base_margin_initialize',
 u'EUREQA_building_block__two-argument_arctangent',
 u'EUREQA_building_block__modulo',
 u'EUREQA_building_block__division',
 u'XGB_scale_pos_weight',
 u'EUREQA_building_block__arcsine',
 u'XGB_tree_method',
 u'EUREQA_building_block__round',
 u'EUREQA_building_block__power',
 u'EUREQA_building_block__logical_not',
 u'EUREQA_building_block__minimum',
 u'EUREQA_max_generations',
 u'EUREQA_building_block__ceiling',
 u'XGB_n_estimators',
 u'EUREQA_building_block__gaussian_function',
 u'EUREQA_building_block__equal-to',
 u'XGB_colsample_bytree',
 u'EUREQA_building_block__inverse_hyperbolic_cosine',
 u'EUREQA_building_block__addition',
 u'XGB_min_child_weight',
 u'XGB_smooth_interval',
 u'EUREQA_building_block__arctangent',
 u'EUREQA_building_block__cosine',
 u'EUREQA_building_block__less-than-or-equal',
 u'EUREQA_building_block__logical_and',
 u'EUREQA_exp

Eureqa does not search for periodic relationships in the data by default.  Doing so would take time away from other types of modeling, so could reduce model quality if no periodic relationships are present.  But let's say we want to check whether Eureqa can find any strong periodic relationships in the data, by allowing it to consider models that use the mathematical <code>sine()</code> function.

In [11]:
tune.set_parameter(
    task_name=task_name,
    parameter_name='EUREQA_building_block__sine',
    value=1)

More values could be set if desired, using the same approach.

Now that some parameters have been set, the tuned model can be run:

In [12]:
job = tune.run()
new_model = job.get_result_when_complete()
new_model

Model(u'Eureqa Generalized Additive Model Classifier (3000 Generations)')

You now have a new model that was run using your specified Advanced Tuning parameters.